In [10]:
import numpy as np
import pandas as pd

In [27]:
df = pd.read_csv('Submission/comments/13i8by9.csv')
df

,Unnamed: 0,ID,Text,Upvotes,Depth,Created Timestamp(UTC),Replies,Author,Post ID
0,0,jk951il,We do have occasional Maldivian posts here. Ju...,19,0,2023-05-15 16:39:32 UTC,0,z80lives,13i8by9
1,1,jk98dy2,"Nop, im from\nMaldives",7,0,2023-05-15 17:05:13 UTC,0,shauts,13i8by9
2,2,jk95l85,no im maldivian,5,0,2023-05-15 16:43:50 UTC,0,Choice-Ad-1394,13i8by9
3,3,jkc5ykd,Need to create a community with a different na...,4,0,2023-05-16 07:09:22 UTC,5,BudovicLagman,13i8by9
4,4,jk8mr7r,"Except for a few thousand others, yes",4,0,2023-05-15 14:03:30 UTC,0,localolala,13i8by9
...,...,...,...,...,...,...,...,...,...
67,67,jkheks8,That's like a whole Maldivian village.,3,2,2023-05-17 10:41:14 UTC,2,BudovicLagman,13i8by9
68,68,jmi92x4,އިޓްސް ދަ އިލޫމިނާޓީ ބްރޯ,1,2,2023-06-01 18:04:25 UTC,0,National-Cut-1128,13i8by9
69,69,js11vdk,I don't know why is said that,1,2,2023-07-15 04:38:40 UTC,0,Maafdndns,13i8by9
70,70,jlpixkv,Lmaoooo,2,3,2023-05-26 15:10:47 UTC,0,Bird_Unusual,13i8by9


#### PREPRocess basic

In [35]:
data1 = df.iloc[:,2:]
data1.head()

,Text,Upvotes,Depth,Created Timestamp(UTC),Replies,Author,Post ID
0,We do have occasional Maldivian posts here. Ju...,19,0,2023-05-15 16:39:32 UTC,0,z80lives,13i8by9
1,"Nop, im from\nMaldives",7,0,2023-05-15 17:05:13 UTC,0,shauts,13i8by9
2,no im maldivian,5,0,2023-05-15 16:43:50 UTC,0,Choice-Ad-1394,13i8by9
3,Need to create a community with a different na...,4,0,2023-05-16 07:09:22 UTC,5,BudovicLagman,13i8by9
4,"Except for a few thousand others, yes",4,0,2023-05-15 14:03:30 UTC,0,localolala,13i8by9


In [36]:
data1.drop('Created Timestamp(UTC)', axis=1,inplace=True)
data1.head()

,Text,Upvotes,Depth,Replies,Author,Post ID
0,We do have occasional Maldivian posts here. Ju...,19,0,0,z80lives,13i8by9
1,"Nop, im from\nMaldives",7,0,0,shauts,13i8by9
2,no im maldivian,5,0,0,Choice-Ad-1394,13i8by9
3,Need to create a community with a different na...,4,0,5,BudovicLagman,13i8by9
4,"Except for a few thousand others, yes",4,0,0,localolala,13i8by9


In [79]:
data1["P(Negative)"] = np.NaN
data1["P(Neutral)"] = np.NaN
data1["P(Positive)"] = np.NaN
data1["Overall"] = np.NaN

In [80]:
data1.head()

,Text,Upvotes,Depth,Replies,Author,Post ID,P(Negative),P(Neutral),P(Positive),Overall
0,We do have occasional Maldivian posts here. Ju...,19,0,0,z80lives,13i8by9,NaN,NaN,NaN,NaN
1,"Nop, im from\nMaldives",7,0,0,shauts,13i8by9,NaN,NaN,NaN,NaN
2,no im maldivian,5,0,0,Choice-Ad-1394,13i8by9,NaN,NaN,NaN,NaN
3,Need to create a community with a different na...,4,0,5,BudovicLagman,13i8by9,NaN,NaN,NaN,NaN
4,"Except for a few thousand others, yes",4,0,0,localolala,13i8by9,NaN,NaN,NaN,NaN


In [47]:
## Loading Model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import logging
logging.set_verbosity_error()
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

In [81]:
for i in range(0,len(data1)):
    text = data1.iloc[i,0]
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    
    ## gives 1D array [Negative, Neutral, Positive]
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    data1.iloc[i,6] = scores[0]
    data1.iloc[i,7] = scores[1]
    data1.iloc[i,8] = scores[2]

    if scores[0] > scores[1] and scores[0] > scores[2]:
        data1.iloc[i,9] = "Negative"
    elif scores[1] > scores[2]:
        data1.iloc[i,9] = "Neutral"
    else:
        data1.iloc[i,9] = "Positive"

In [82]:
data1

,Text,Upvotes,Depth,Replies,Author,Post ID,P(Negative),P(Neutral),P(Positive),Overall
0,We do have occasional Maldivian posts here. Ju...,19,0,0,z80lives,13i8by9,0.048782,0.589844,0.361373,Neutral
1,"Nop, im from\nMaldives",7,0,0,shauts,13i8by9,0.079881,0.791347,0.128772,Neutral
2,no im maldivian,5,0,0,Choice-Ad-1394,13i8by9,0.120242,0.790698,0.089060,Neutral
3,Need to create a community with a different na...,4,0,5,BudovicLagman,13i8by9,0.078106,0.717654,0.204240,Neutral
4,"Except for a few thousand others, yes",4,0,0,localolala,13i8by9,0.152903,0.786789,0.060309,Neutral
...,...,...,...,...,...,...,...,...,...,...
67,That's like a whole Maldivian village.,3,2,2,BudovicLagman,13i8by9,0.325326,0.577492,0.097182,Neutral
68,އިޓްސް ދަ އިލޫމިނާޓީ ބްރޯ,1,2,0,National-Cut-1128,13i8by9,0.271838,0.663358,0.064804,Neutral
69,I don't know why is said that,1,2,0,Maafdndns,13i8by9,0.660334,0.323360,0.016306,Negative
70,Lmaoooo,2,3,0,Bird_Unusual,13i8by9,0.204805,0.492023,0.303171,Neutral
